<h2 align="center"><b>ALINHAMENTO E GERAÇÃO DE ÁRVORE FILOGENÉTICA</b></h2>
<h3 align="center"><b>IMPORTS E CONFIGURAÇÃO DO DIRETÓRIO BASE</b></h3>
<p align="center">IMPORTANTE LEMBRAR QUE É NECESSÁRIO TER O CLUSTALW INSTALADO PARA RODAR O CÓDIGO</p>

In [2]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO, Phylo
from Bio import SeqIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import os

# Substitua pelo caminho do diretório que você deseja listar
input_path = 'testset'
diretorio = '../data/{input_path}/'.format(input_path = input_path)
arquivos = os.listdir(diretorio)

<h3 align="center"> <b>LIMPEZA</b> </h3>

In [4]:

def clean_Trees():
    dir_Trees = '../data/out/Trees/' 
    arquivos_trees = os.listdir(dir_Trees)

    for name_file_trees in arquivos_trees:
        path_trees = dir_Trees + "{name}".format(name = name_file_trees)
        if name_file_trees != "file.gitkeep":
            os.remove(path_trees)

def clean_tmp():
    dir_tmp = '../data/out/tmp/' 
    arquivos_tmp = os.listdir(dir_tmp)

    for name_file_tmp in arquivos_tmp:
        path_tmp = dir_tmp + "{name}".format(name = name_file_tmp)
        if name_file_tmp != "file.gitkeep":
            os.remove(path_tmp)


def clean_NoPipe():
    dir_path = '../data/{input_path}/'.format(input_path = input_path)

    for file_name in os.listdir(dir_path):
        if 'NoPipe' in file_name:
            file_path = os.path.join(dir_path, file_name)
            os.remove(file_path)

clean_NoPipe()
clean_tmp()
clean_Trees()

<h3 align="center"> <b>VALIDADORES</b> </h3>


In [3]:

#Função que verifica se todas as sequências são proteínas válidas no formato FASTA"""
def validar_sequencias(arquivo):
    with open(arquivo, 'r') as f:
        for linha in f:
            if linha.startswith('>'):
                continue  # Pula a linha de cabeçalho
            sequencia = linha.strip()
            if not set(sequencia).issubset(set('ACDEFGHIKLMNPQRSTVWY')):
                return False
    return True


def tem_nomes_duplicados(nome):
    registros = SeqIO.parse(nome, 'fasta')
    nomes = set()
    for registro in registros:
        nome = registro.id
        if nome in nomes:
            return True
        nomes.add(nome)
    return False

<h3 align="center"> <b>TRATAMENTO DE SEQUENCIA</b> </h3>


In [4]:
def remove_pipe(name, path_in_fasta):

    sequences = list(SeqIO.parse(path_in_fasta, "fasta"))

    # Criar um dicionário para armazenar as sequências únicas
    unique_sequences = {}

    # Iterar pelas sequências do arquivo de entrada
    for sequence in sequences:
        # Verificar se a sequência já existe no dicionário de sequências únicas
        if str(sequence.seq) not in unique_sequences:
            # Se a sequência é única, armazená-la no dicionário
            unique_sequences[str(sequence.seq)] = sequence

    # Criar uma lista de sequências únicas
    unique_sequences_list = list(unique_sequences.values())

    # Salvar as sequências únicas em um arquivo de saída
    output_file_tmp = "sequencias_sem_duplicatas.fasta"
    SeqIO.write(unique_sequences_list, output_file_tmp, "fasta")

    # Mover o arquivo de saída .dnd para o diretório "resultados"
    result = "../data/{input_path}/{name}_NoPipe".format(input_path = input_path, name=name)

    os.rename(output_file_tmp, result)
    return result
    

<h3 align="center"> <b>CONSTRUTUOR</b> </h3>


In [5]:
# Mais informações sobre aplicações biopython e clustalwcommandline - https://biopython.org/docs/1.76/api/Bio.Align.Applications.html
def constructor_tree(path_in_fasta, path_out_aln, path_out_dnd, path_old_dnd, path_out_tree):
    # Executa o Clustalw para alinhar as sequências
    clustalw_cline = ClustalwCommandline("clustalw", infile=path_in_fasta, outfile=path_out_aln)# Executa o programa clustalw sem precisar da linha de comando.
    clustalw_cline()#  Executa o comando ClustalW com base nos parâmetros definidos no objeto ClustalwCommandline e retorna os resultados da execução na forma de uma tupla de strings.

    # Mover o arquivo de saída .dnd para o diretório "resultados"
    os.rename(path_old_dnd, path_out_dnd)

    '''
    Clustalw_cline() - gera 2 arquivos de saida por padrão

    Nesse caso:
    - ORTHOMCL256.aln: Contendo a sequência de ORTHOMCL256 alinhada em formato clustal
    - ORTHOMCL256.dnd: Contendo informações sobre o agrupamento hierárquico das sequências alinhadas.
    '''

    # Abre o arquivo de alinhamento
    with open(path_out_aln, "r") as handle:
        alignment = AlignIO.read(handle, "clustal")# O objeto MultipleSeqAlignment retornado é armazenado na variável.

    # Calcula a matriz de distância
    # argumento 'identity', que indica que a distância entre as sequências será medida pelo número de identidades, ou seja, a fração de posições nas sequências que possuem o mesmo nucleotídeo ou aminoácido.
    calculator = DistanceCalculator('identity') 
    # Calcula a matriz de distâncias entre as sequências
    distance_matrix = calculator.get_distance(alignment) 

    # Constrói a árvore filogenética
    # constr árvores filogenéticas a partir de matrizes de distâncias entre sequências.
    constructor = DistanceTreeConstructor() 
    tree = constructor.nj(distance_matrix)

    # Salva a árvore em formato nexus
    Phylo.write(tree, path_out_tree, "nexus")

    # Salva a árvore em formato newick
    # IMPORTANTE MUDAR A EXTENSÃO DO path_out_tree
    # Phylo.write(tree, path_out_tree, "newick")

<h3 align="center"> <b>PERCORRER E MANIPULA DIRETORIO</b> </h3>


In [6]:
for name_file in arquivos:
    #configurando caminhos relativos padrao do diretorio
    path_in_fasta = "../data/{input_path}/{name}".format(input_path = input_path, name=name_file)
    path_out_aln = "../data/out/tmp/{name}.aln".format(name=name_file)
    path_out_dnd = "../data/out/tmp/{name}.dnd".format(name=name_file)
    path_old_dnd = "../data/{input_path}/{name}.dnd".format(input_path = input_path, name=name_file)
    #Salva em newick, para salvar em nexus mude o parâmetro da funçao constructor_tree para path_out_tree_nexus
    path_out_tree_nwk = "../data/out/Trees/tree_{name}.nwk".format(name=name_file)
    #Salva em nexus, para salvar em nexus mude o parâmetro da funçao constructor_tree para path_out_tree_nwk
    path_out_tree_nexus = "../data/out/Trees/tree_{name}.nexus".format(name=name_file)
    path_out_maf = "../data/out/Maf/{name}.maf".format(name=name_file)

    if not(tem_nomes_duplicados(path_in_fasta)) and validar_sequencias(path_in_fasta):
        constructor_tree(path_in_fasta, path_out_aln, path_out_dnd, path_old_dnd, path_out_tree_nexus)
    else:
        path_in_fasta = remove_pipe(name_file, path_in_fasta)
        path_old_dnd = "{path}.dnd".format(path=path_in_fasta)

        constructor_tree(path_in_fasta, path_out_aln, path_out_dnd, path_old_dnd, path_out_tree_nexus)

<h3 align="center"> <b>Limpeza arquivos temporários</b> </h3>


In [5]:
clean_NoPipe()
clean_tmp()